In [1]:
import os.path
import csv
import time
from time import sleep
import random
import pandas as pd
import requests
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt



from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
from parsel import Selector


#Imports for automation without proxycurl

from bs4 import BeautifulSoup
import re

#To find additional xpaths
#https://www.linkedin.com/pulse/techniques-find-xpath-selenium-webdriver-vivekanand-r-yadav/

In [2]:
# first part is getting the links using automation

scrolling = False

def scrolldown(driver, timeout = 20, sleep_interval = 5):
    """A method for scrolling the page within a timeout."""
    start_time = time.time()
    scroll_count = 0
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to the bottom.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        scroll_count += 1
        # Wait to load the page.
        time.sleep(sleep_interval)
        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("Reached end of page.")
            break
        last_height = new_height
        # Check if timeout is reached
        if time.time() - start_time > timeout:
            print("Timeout reached.")
            break

    print(f"Scroll count: {scroll_count}")

In [3]:
# skipping this

# defining a function called login_linkedin

def login_linkedin(file_name, driver, login_url = 'https://www.linkedin.com/login'):
    # Open the file with context manager to ensure it's closed properly after reading
    with open(file_name, 'r') as credential:
        uname = credential.readline().strip()  # Read the first line for username
        password = credential.readline().strip()  # Read the second line for password

    print('- Finish importing the login credentials')
    
   
    # Navigate to the LinkedIn login page
    driver.get(login_url)

    try:
        # Wait for the username field to become available and enter the username
        username = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "username"))
        )
        
        time.sleep(2)
        username.send_keys(uname)

        # Wait for the password field to become available and enter the password
        pword = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "password"))
        )
        
        time.sleep(2)
        pword.send_keys(password)

        # Wait for the submit button to become clickable and click it
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']"))
        )
        
        time.sleep(2)
        login_button.click()

        print('Login successful')
    except (NoSuchElementException, TimeoutException) as e:
        print(f'An error occurred during login: {e}')

In [4]:
# basic driver function. 

def go_to_URL(driver, URL, scrolling = False, scroll_timeout = 0):
    try:
        driver.get(URL)
        # Wait for the necessary page elements to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        
        # If scrolling is required, use the scrolldown function
        if scrolling:
            scrolldown(driver, timeout=scroll_timeout)
            print("Alumni Block after scrolling")
    except WebDriverException as e:
        print(f"Failed to navigate to {URL} or an error occurred while scrolling: {e}")
        
        
        

In [5]:
# call the function and loads the csv file. Loads the csv file of links
# https://medium.com/@udayshadani/errors-we-can-get-during-web-scraping-and-how-to-handle-it-41de0c2f6ec3

def load_csv(file_name):
    try:
        return pd.read_csv(file_name)
    except FileNotFoundError:
        print('File not found error')
        



In [6]:
def validate_fields(field):
    if field:
        pass
    else:
        field = 'NO Results'
    return field

from lxml import html

def extract_from_html(html_snippet):
    # Parse the HTML content of the snippet
    tree = html.fromstring(html_snippet)
    
    # Use XPath to select the text within the <h1> tag
    name = tree.xpath("//h1/text()")
    
    return name[0].strip() if name else None


In [7]:
#new updated function
#3672 seba bus_dev
#5587 consulting

def automation_links(driver, csv_data, numberlinks = 2, start = 6948):
    Jobdata = []
    experience = []
# its going to start and stop at this number
    for i in csv_data['url'][start:start + numberlinks]:
        #driver.get('https://www.linkedin.com/in/romaxavierm/')  # To get the profile URL
        driver.get(i)
        time.sleep(5)
        
            
        # Get education
        soup = BeautifulSoup(driver.page_source, 'lxml')  # Using BeautifulSoup to parse the HTML

        # Find all <section> elements
        section_list = soup
        #soup.find_all("section")

        # Define a function to safely extract text using BeautifulSoup
        def extract_text_by_xpath(soup, xpath):
            elements = soup.select(xpath)
            if elements:
                return elements[0].get_text(strip=True)
            return 'None'
        # First, locate the experience section anchor
        expAnchor = soup.find('div', id ='experience')
        j = soup.select('section:has(#experience)>div>ul>li') # company element
        company_list=[]
        title_list=[]
        extra_list=[]
        for x in j:
            #mr1.t-bold span[aria-hidden="true"]
            title = x.select_one('div[class="display-flex align-items-center mr1 t-bold"] span') # title element
            if title == None:
                title =''
                extra=''
            else:
                title=title.text
                temp= x.select('span[class="t-14 t-normal t-black--light"] span')
                extra = [tt.text for tt in temp]
            jj = x.select('span[class="t-14 t-normal"] span')
            jjj = [jj_item.text for jj_item in jj]
            company_list.append(jjj[0])
            title_list.append(title)
            extra_list.append(extra[::2]) 
            
        next_siblings = expAnchor.find_next_siblings()

        # Concatenate the HTML of all these siblings
        siblings_html = ''.join(str(sibling) for sibling in next_siblings)

        # Parse this concatenated HTML into a new BeautifulSoup object
        experience_section = BeautifulSoup(siblings_html, 'html.parser')

        
        # Define the Cascading Style Sheets selectors corresponding to the XPaths provided
        name_selector = "a.pv-text-details__about-this-profile-entrypoint h1"
        #expAnchor = soup.select('div.pvs-list__outer-container')
        #expAnchor = soup.select('div#experience div.pvs-list__outer-container')
            
        company_selector = 'div.pvs-entity--padded span.t-14.t-normal span[aria-hidden="true"]'
        job_title_selector = 'div.display-flex.align-items-center.mr1.t-bold span[aria-hidden="true"]'
        university_name_selector = "section.artdeco-card h2 span[aria-hidden='true']:contains('Education') ~ div.display-flex align-items-center mr1 hoverable-link-text t-bold span[aria-hidden='true']"
        degree_selector = "section.artdeco-card h2 span[aria-hidden='true']:contains('Education') ~ div.pvs-list__outer-container li a.optional-action-target-wrapper.display-flex.flex-column.full-width span.t-14 span[aria-hidden='true']"
        dates_selector = "section.artdeco-card h2 span[aria-hidden='true']:contains('Education') ~ div.pvs-list__outer-container li a.optional-action-target-wrapper.display-flex.flex-column.full-width span.t-black--light span[aria-hidden='true']"
            
        #location_selector = "div.pv-text-details__left-panel.mt2 span.text-body-small.inline.t-black--light.break-words"
        #email_selector = "div.pv-contact-info__ci-container a[href^='mailto:']"
            
        # Now you can use CSS selectors on experience_section
        company_elements = experience_section.select(company_selector)
        job_title_elements = experience_section.select(job_title_selector)



        # Extract data using BeautifulSoup selectors
        name = extract_text_by_xpath(soup, name_selector)
        companies = [company.get_text().strip() for company in company_elements]
        job_titles = [title.get_text().strip() for title in job_title_elements]
        #location = extract_text_by_xpath(soup, location_selector)
        #email = extract_text_by_xpath(soup, email_selector)

        # Validate fields if necessary (assuming you have a validate_fields function)
        name = validate_fields(name)
        companies = validate_fields(companies)
        job_titles = validate_fields(job_titles)
        #location = validate_fields(location)
        #email = validate_fields(email)
            
        # Assuming 'soup' is your BeautifulSoup object containing the parsed HTML
        colleges = []
        degrees = []  # List to store degree names
        dates_list = []  # List to store dates

        # Selectors targeting a more specific range within the education section
        college_elements = soup.select('section.artdeco-card li a[href*="linkedin.com/company"] div.display-flex.align-items-center.mr1.hoverable-link-text.t-bold span[aria-hidden="true"]')
        degree_elements = soup.select('section.artdeco-card li a[href*="linkedin.com/company"] span.t-14 span[aria-hidden="true"]')
        date_elements = soup.select('section.artdeco-card li a[href*="linkedin.com/company"] span.t-black--light span[aria-hidden="true"]')

        # Regular expression to match strings containing digits
        digit_pattern = re.compile(r'\d')
            
                
        for college in college_elements:
            text = college.get_text(strip=True)
            # Check if the text contains "university" or "college" (case-insensitive)
            if "university" in text.lower() or "college" in text.lower():
                colleges.append(text)

        for degree in degree_elements:
            text = degree.get_text(strip=True).lower()  # Convert to lower case for case-insensitive comparison
            if (not digit_pattern.search(text) 
                and 'followers' not in text
                and 'ca' not in text
                and 'california' not in text
                and 'time' not in text):
                degrees.append(text.title())  # Convert back to title case for display

        for date in date_elements:
            text = date.get_text(strip=True)
            if digit_pattern.search(text) and 'followers' not in text.lower() and 'mos' not in text.lower():  # Include text with digits but exclude 'followers' and 'mos'
                dates_list.append(text)
                    
        # Filter out empty or "Unknown" job titles and companies
        job_titles = [title for title in job_titles if title.strip() not in ('', 'Unknown')]
        companies = [company for company in companies if company.strip() not in ('', 'Unknown')]

        # Exclude entries that contain common keywords that are not job titles or companies
        exclude_keywords = ['followers', 'education', 'members', 'published', '·']
        job_titles = [title for title in job_titles if all(keyword not in title.lower() for keyword in exclude_keywords)]
        companies = [company for company in companies if all(keyword not in company.lower() for keyword in exclude_keywords)]

        # Check the length of job titles and companies to exclude very short or suspicious entries
        min_length = 2  # Minimum length for job titles and companies
        job_titles = [title for title in job_titles if len(title.split()) >= min_length]
        companies = [company for company in companies if len(company.split()) >= min_length]
            


        # Printing the outputs
        print('\n')
        print('Name:', name)
        print('Link:', i)
        print('Experience: ', title_list)
        print('Employers: ', company_list)
        print('Colleges: ', colleges)
        print('Degrees: ', degrees)
        print('Dates Attended: ', dates_list)
        print('Extra Info: ', extra_list)

        # Storing data in a dictionary
        data = {
            'Name': name,
            'url': i,
            'Experience': title_list,
            'Employers': company_list,
            'Colleges': colleges,
            'Degrees': degrees,
            'Dates Attended': dates_list,
            'Extra Info:': extra_list
            }
        Jobdata.append(data)
        

    # Exporting this into an Excel file
    df = pd.DataFrame(Jobdata)
    return df

In [8]:
# logs you into linkedin

# Call all functions to go get links

opts = Options()

# using my useragent?

opts.add_argument("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")
                                                               
#opts.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0")

driver = webdriver.Chrome(options = opts)

#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

Login_URL = r"https://linkedin.com/uas/login"
driver.get(Login_URL)

# waiting for page to load
sleep(3)

file = "login_credential.txt"

login_linkedin(file, driver)

Saint_Marys_URL = r"https://www.linkedin.com/school/saint-mary's-college-of-california/people/"

go_to_URL(driver, Saint_Marys_URL)



- Finish importing the login credentials
Login successful


In [9]:
# setting up the links to use GrowMeOrganic Code to create the database.

csv_name = 'links.csv'

csv_data = load_csv(csv_name)

print(csv_data)

df2 = automation_links(driver, csv_data)



                                                    url
0     https://www.linkedin.com/in/ACwAAABvQKABkFKBAu...
1     https://www.linkedin.com/in/ACwAAAER0qIBV5XZpx...
2     https://www.linkedin.com/in/ACwAAATJ8wsBMoEv2e...
3     https://www.linkedin.com/in/ACwAAAYgqzIBjNJ8iV...
4     https://www.linkedin.com/in/ACwAAAg8-XoBNsL5in...
...                                                 ...
8345  https://www.linkedin.com/in/ACwAAANE6h0BacqAyI...
8346  https://www.linkedin.com/in/ACwAAAa2Db0BmhhxJL...
8347  https://www.linkedin.com/in/ACwAAB-n5GUBfwG9mz...
8348  https://www.linkedin.com/in/ACwAAEBtS2UBkGyUwV...
8349  https://www.linkedin.com/in/ACwAAEFd3dkB5saXfs...

[8350 rows x 1 columns]


Name: Jon (Jonathan) Gottfried
Link: https://www.linkedin.com/in/ACwAAAAG9L8B1gDGO8-lNyM_WXd9fY4N85xAcLQ
Experience:  ['CFO/COO', 'VP', 'Principal', 'Validation Manager', 'Validation Supervisor']
Employers:  ['TekTeam · Full-time', 'AQS', 'GW Associates', 'Pharmtech', 'VSI']
Colleges:  ['St Mary’s Un

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.199)
Stacktrace:
0   chromedriver                        0x0000000105fc4d28 chromedriver + 4795688
1   chromedriver                        0x0000000105fbc2b3 chromedriver + 4760243
2   chromedriver                        0x0000000105b9588d chromedriver + 407693
3   chromedriver                        0x0000000105b69758 chromedriver + 227160
4   chromedriver                        0x0000000105c1150f chromedriver + 914703
5   chromedriver                        0x0000000105c270e6 chromedriver + 1003750
6   chromedriver                        0x0000000105c0ba73 chromedriver + 891507
7   chromedriver                        0x0000000105bd6143 chromedriver + 672067
8   chromedriver                        0x0000000105bd731e chromedriver + 676638
9   chromedriver                        0x0000000105f85795 chromedriver + 4536213
10  chromedriver  

In [ ]:
# Saved the database to the csv file.

# Specify the filename

print(df2)

filename = 'new_36.csv'

# Write the DataFrame to a .csv file
df2.to_csv(filename, index = False, encoding ='utf-8')

print(f"Data has been written to {filename}")